# Agents

## 1.静态模型

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    "openai:gpt-5",
    tools=[]
)

In [ ]:
from langchain_openai import ChatOpenAI
from dataclasses import dataclass
from langchain.agents import create_agent

model = ChatOpenAI(
    model="gpt-5",
    temperature=0.7,
     max_tokens=4096,
    timeout=30,
)

@dataclass
class ResponseFormat:
    answer: str

agent = create_agent(
    model=model,
    system_prompt="You are a concise assistant. Answer clearly and briefly.",
    response_format=ResponseFormat
    # tools=[]
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "人生的意义是什么?"}]}
)
print(response)

In [ ]:
print(list(response)) # or use print(response.keys())
print(response["structured_response"].answer)
print(response['messages'])  # 转换为标准字典

print(dict(response))


## 2.动态模型

In [ ]:
"""
创建一个智能体，它会在对话早期使用轻量、便宜的 gpt-4.1-mini，当对话变长时自动切换到更强大的 gpt-5。这样既能节省成本，又能保证复杂任务的质量。
"""
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.tools import tool, ToolRuntime
from dataclasses import dataclass


basic_model = ChatOpenAI(model="gpt-4o-mini")
advanced_model = ChatOpenAI(model="gpt-5")

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])

    if message_count > 10:
        # Use an advanced model for longer conversations
        model = advanced_model
    else:
        model = basic_model

    request.model = model
    return handler(request)

@dataclass
class ResponseFormat:
    answer: str

agent = create_agent(
    model=basic_model,  # Default model
    tools=[],
    system_prompt="You are a helpful assistant.",
    middleware=[dynamic_model_selection],
    response_format=ResponseFormat,
)

response = agent.invoke({"messages": [{"role": "user", "content": "如何学习 LangChain V1.0.0?"}]})



In [ ]:
print(response["structured_response"].answer)

## 3.工具

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_tool_call
from langchain.messages import ToolMessage


@tool
def search(query: str) -> str:
    """Search for information."""
    return f"Results for: {query}"

@tool
def get_weather(location: str) -> str:
    """Get weather information for a location."""
    if not location or location.strip() == "":
        raise ValueError("Missing required parameter: location")
    return f"Weather in {location}: Sunny, 72°F"

# 自定义如何处理工具错误
@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )
# ⚡ 关键点：自定义 system prompt
system_prompt = """You are a helpful AI assistant.
You have access to tools.
If the user input is ambiguous or missing required parameters, DO NOT ask for clarification.
Instead, call the tool anyway with empty or default values.
"""

agent = create_agent(
    model="openai:gpt-4o",
    tools=[search, get_weather],
    middleware=[handle_tool_errors],
    system_prompt=system_prompt
)

response = agent.invoke({
    "messages": [
        {"role": "user", "content": "What's the weather?"}
    ]
})

print(response) 


## System prompt

In [ ]:
agent = create_agent(
    model,
    tools,
    system_prompt="You are a helpful assistant. Be concise and accurate."
)

In [ ]:
# 动态系统提示
from typing import TypedDict

from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest


class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    user_role = request.runtime.context.get("user_role", "user")
    base_prompt = "You are a helpful assistant."

    if user_role == "expert":
        return f"{base_prompt} Provide detailed technical responses."
    elif user_role == "beginner":
        return f"{base_prompt} Explain concepts simply and avoid jargon."

    return base_prompt

agent = create_agent(
    model="openai:gpt-4o",
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context
)

# The system prompt will be set dynamically based on context
result = agent.invoke(
    {"messages": [{"role": "user", "content": "解释机器学习"}]},
    context={"user_role": "beginner"}
)

print(result)

In [ ]:
print(result["messages"])
ai_message = result["messages"][1]
print(ai_message.content)

## Invocation 调用

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather in San Francisco?"}]}
)

## Structured output 结构化输出

In [ ]:

# ToolStrategy

from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

## ToolStrategy 解释说明

`ToolStrategy(ResponseFormat)` 是 `@dataclass ResponseFormat` 的增强版

### 两者的本质区别
| 对比项        | `ToolStrategy(ContactInfo)`            | `@dataclass ResponseFormat`       |
| ---------- | -------------------------------------- | --------------------------------- |
| **定位**     | LangChain 新版 **“工具策略（Tool Strategy）”** | 传统 **结构化输出模式（Structured Output）** |
| **主要作用**   | 控制模型调用工具的输入/输出格式                       | 约束模型的最终输出结构                       |
| **典型使用场景** | Agent 调用多个工具，返回值结构化                    | 模型输出固定 JSON 样式的结果                 |
| **底层语义**   | 模型需遵循工具协作协议                            | 模型需直接生成符合 schema 的 JSON           |
| **解析逻辑**   | 由 `ToolStrategy` 处理工具调用的 I/O           | 由 `response_format` 解析为对象或字典      |

### 底层差异总结
| 层级           | ToolStrategy     | Dataclass   |
| ------------ | ---------------- | ----------- |
| 调用层          | Agent 工具策略层      | 模型输出解析层     |
| 语义           | “如何调用工具并返回结构化数据” | “如何生成结构化输出” |
| 是否需要工具       | ✅ 可有工具           | ❌ 通常无工具     |
| LangChain 版本 | 0.2+ 新版推荐写法      | 旧版（兼容但简化）   |
| 扩展性          | 可定义多工具行为策略       | 仅定义单输出结构    |

### 企业最佳实践建议
| 场景                         | 推荐写法                           |
| -------------------------- | ------------------------------ |
| 智能体调用多个 API / 工具，需要统一返回结构  | ✅ `ToolStrategy(BaseModel)`    |
| 只是需要模型输出固定 JSON 结构（如总结、提取） | ✅ `@dataclass response_format` |
| 需要可扩展策略（错误处理、参数检查、可控调用）    | ✅ `ToolStrategy` 更优            |



## Memory

状态中存储的信息可以被视为代理的短期记忆

定义自定义状态有两种方法:
- 通过中间件 （推荐）
- 通过 state_schema 在 create_agent 上

In [ ]:
# 通过中间件

from langchain.agents import AgentState
from langchain.agents.middleware import AgentMiddleware
from langchain.tools import tool
from langchain.agents import create_agent


class CustomState(AgentState):
    user_preferences: dict

class CustomMiddleware(AgentMiddleware):
    state_schema = CustomState
    tools = []

    def before_model(self, state: CustomState, runtime):
        """
        在模型调用前对消息做预处理。
        """
        prefs = state.get("user_preferences", {})

        # 在对话前插入一条系统提示，指引模型如何回答
        if prefs:
            style = prefs.get("style", "neutral")
            verbosity = prefs.get("verbosity", "normal")
            instruction = f"回答风格：{style}；详细程度：{verbosity}。"
            return {
                "messages": state["messages"] + [{"role": "system", "content": instruction}]
            }
        return None

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    middleware=[CustomMiddleware()]
)

# The agent can now track additional state beyond messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "请介绍一下 LangChain"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

print(result['messages'][1].content) # SystemMessage
print(result['messages'][2].content) # AIMessage

In [ ]:
# 使用 state_schema 参数作为快捷方式来定义仅在工具中使用的自定义状态。

from langchain.agents import AgentState
from langchain.tools import tool
from langchain.agents import create_agent

@tool
def search(query: str) -> str:
    """Search for a query."""
    return f"Search results for {query}"

class CustomState(AgentState):
    user_preferences: dict

agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search],
    state_schema=CustomState
)
# The agent can now track additional state beyond messages
result = agent.invoke({
    "messages": [{"role": "user", "content": "介绍 LangChain"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

print(result)

## Streaming

In [ ]:
from langchain.agents import create_agent

def search_ai_news(query: str) -> str:
    """Search for AI news articles."""
    return f"AI news about {query}"

agent = create_agent(
    model="openai:gpt-5-nano",
    tools=[search_ai_news],
)
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}", end="", flush=True)
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}", end="", flush=True)





In [ ]:
from langchain.chat_models import init_chat_model

# 初始化模型
model = init_chat_model("openai:gpt-5-nano")

# 直接流式打印 token
for chunk in model.stream("Explain the future of AI in 5 steps."):
    print(chunk.content, end="", flush=True)


## 3.1 传递配置好的 ToolNode

In [ ]:
from langchain.agents import ToolNode

tool_node = ToolNode(
    tools=[search, calculate],
    handle_tool_errors="Please check your input and try again."
)
agent = create_agent(model, tools=tool_node)
result = agent.invoke({"messages": [...]})

## 4.prompt

In [ ]:
agent = create_agent(
    model,
    tools,
    prompt="You are a helpful assistant. Be concise and accurate."
)

## 高级配置 -- 结构化输出

In [ ]:
from pydantic import BaseModel
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model,
    tools=[search_tool],
    response_format=ContactInfo
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

## 高级配置 -- 记忆

In [ ]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph.message import add_messages
from langchain.agents import create_agent
from langchain.agents import AgentState

class CustomAgentState(AgentState):
    messages: Annotated[list, add_messages]
    user_preferences: dict

agent = create_agent(
    model,
    tools=tools,
    state_schema=CustomAgentState
)

# The agent can now track additional state beyond messages. This custom state can be accessed and updated throughout the conversation.
result = agent.invoke({
    "messages": [{"role": "user", "content": "I prefer technical explanations"}],
    "user_preferences": {"style": "technical", "verbosity": "detailed"},
})

## 高级配置 -- 模型前钩子

In [ ]:
from langchain_core.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.agents import create_agent

def trim_messages(state):
    """Keep only the last few messages to fit context window."""
    messages = state["messages"]

    if len(messages) <= 3:
        return {"messages": messages}

    first_msg = messages[0]
    recent_messages = messages[-3:] if len(messages) % 2 == 0 else messages[-4:]
    new_messages = [first_msg] + recent_messages

    return {
        "messages": [
            RemoveMessage(id=REMOVE_ALL_MESSAGES),
            *new_messages
        ]
    }

agent = create_agent(
    model,
    tools=tools,
    pre_model_hook=trim_messages
)

## 高级配置 --  模型后钩子

In [ ]:
from langchain_core.messages import AIMessage, RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES

def validate_response(state):
    """Check model response for policy violations."""
    messages = state["messages"]
    last_message = messages[-1]

    ## confidential 机密
    if "confidential" in last_message.content.lower():
        return {
            "messages": [
                RemoveMessage(id=REMOVE_ALL_MESSAGES),
                *messages[:-1],
                AIMessage(content="I cannot share confidential information.")
            ]
        }

    return {}

agent = create_agent(
    model,
    tools=tools,
    post_model_hook=validate_response
)

## 高级配置 -- 流式传输

In [ ]:
for chunk in agent.stream({
    "messages": [{"role": "user", "content": "Search for AI news and summarize the findings"}]
}, stream_mode="values"):
    # Each chunk contains the full state at that point
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")

In [ ]:
## 流失输出 demo

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import create_agent


# 1. 定义一个最简单的工具
@tool
def search_ai_news(query: str) -> str:
    """Search the latest AI news (dummy implementation)."""
    return f"Found some dummy AI news for query: {query}"


# 2. 定义模型（这里用 OpenAI）
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 3. 创建 agent
agent = create_agent(
    model=model,
    tools=[search_ai_news]
)

# 4. 运行并流式打印结果
for chunk in agent.stream(
    {
        "messages": [
            {"role": "user", "content": "Search for AI news and summarize the findings"}
        ]
    },
    stream_mode="values",
):
    # 每个 chunk 是当前状态
    latest_message = chunk["messages"][-1]
    if latest_message.content:
        print(f"Agent: {latest_message.content}")
    elif latest_message.tool_calls:
        print(f"Calling tools: {[tc['name'] for tc in latest_message.tool_calls]}")
